In [1]:
!pwd

/kaggle/working


## Import libraries

In [117]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import torch
from torch.utils.data import TensorDataset, DataLoader


## Import data

Import data, scale appropriately and convert to numpy array

In [6]:
milk_df = pd.read_csv('../input/milk-classification/milk.csv')

In [8]:
print(milk_df.columns)
mean_milk = milk_df.mean(numeric_only=True)
std_milk = milk_df.std(numeric_only=True)
max_milk = milk_df.max()
min_milk = milk_df.min()
nunique_milk = milk_df.nunique()

Index(['pH', ' temperature', ' taste', ' odor', ' fat', ' turbidity',
       ' colour', ' grade'],
      dtype='object')


In [9]:
# enc = OneHotEncoder(categories = ['medium_quality','low_quality','high_quality'],handle_unknown='error')
enc = OneHotEncoder()

In [10]:
scaled_pH = (milk_df['pH']-6.630123)/1.399679
scaled_temperature = (milk_df[' temperature']-44.22663)/10.09836
scaled_taste = milk_df[' taste']
scaled_odor = milk_df[' odor']
scaled_fat = milk_df[' fat']
scaled_turbidity = milk_df[' turbidity']
scaled_color = (milk_df[' colour']-251.8404)/4.307424
scaled_grade = enc.fit_transform(milk_df[[' grade']])

In [11]:
X = np.array([scaled_pH, scaled_temperature, scaled_taste, scaled_odor, scaled_fat, scaled_turbidity, scaled_color]).T.astype(np.float32)
Y = scaled_grade.A

In [88]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [89]:

batch_size = 50
dataset_train = DataLoader(TensorDataset(torch.tensor(x_train), torch.tensor(y_train)), batch_size=batch_size, shuffle=True)
dataset_test = DataLoader(TensorDataset(torch.tensor(x_test), torch.tensor(y_test)), batch_size=1, shuffle=True)

## Train Neural Net

Construct a simple ff neuralNet. And train on it.

In [111]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 5)
        self.fc2 = nn.Linear(5, 3)
        
    def forward(self, x):
        x = self.fc1(x)
        # x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        # x = F.softmax(x)
        
        return x
    
net = Net()

In [49]:
num_epochs = 500
learning_rate = 0.01

In [112]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [113]:
for epoch in range(num_epochs):
    # Mini batch learning
    for x_train, y_train in dataset_train:
        # Forward + Backward + Optimize        
        y_pred = net(x_train)
        loss = criterion(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print('Epoch {}'.format(epoch+1), ':: loss', np.around(loss.item(),4))


Epoch 1 :: loss 0.8948
Epoch 51 :: loss 0.044
Epoch 101 :: loss 0.0075
Epoch 151 :: loss 0.0046
Epoch 201 :: loss 0.0025
Epoch 251 :: loss 0.0008
Epoch 301 :: loss 0.0017
Epoch 351 :: loss 0.0003
Epoch 401 :: loss 0.0003
Epoch 451 :: loss 0.0001


In [114]:
count = 0
for eg_idx, (x_test, y_test) in enumerate(dataset_test):
    _y_pred = F.softmax(net(x_test), dim=-1)
    y_pred = torch.argmax(_y_pred, dim = -1)
    count += y_test[0,y_pred[0]]

In [115]:
count.item(),(eg_idx+1)

(211.0, 212)

## Random Forest Classfier

In [144]:
clf = RandomForestClassifier(max_depth=20, random_state=0)
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, random_state=0)

In [145]:
count = 0
for eg_idx, (x_test, y_test) in enumerate(dataset_test):
    y_pred = clf.predict(x_test)
#     print(eg_idx, type(y_pred), type(y_test), y_test, np.argmax(y_pred))
    if y_test[0][np.argmax(y_pred)] == 1.:
        count += 1

In [146]:
count,(eg_idx+1)

(202, 212)